In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import keras
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam, Nadam
from keras.preprocessing import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
trainvar = pd.read_csv('training_variants')
traintex = pd.read_csv('training_text', sep='\|\|', header=None, names=['ID','Text'], skiprows=[0])
train = pd.merge(trainvar, traintex, on='ID')

testvar = pd.read_csv('test_variants')
testtex = pd.read_csv('test_text', sep='\|\|', header=None, names=['ID','Text'], skiprows=[0])
test = pd.merge(testvar, testtex, on='ID')

/home/arun/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/arun/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
train.Text = train.Gene+' '+train.Variation+' '+train.Text
test.Text = test.Gene+' '+test.Variation+' '+test.Text

In [5]:
y = keras.utils.to_categorical(train['Class']-1, num_classes=9)
max_length=10000
s_words = ['well','determine']
vector = CountVectorizer(ngram_range=(3,3),stop_words=stopwords.words('english').append(s_words), min_df=5, max_features=max_length)
X = vector.fit_transform(train.Text,train.Class-1)
tran = TfidfTransformer()
X = tran.fit_transform(X,train.Class-1)
X.shape

(3321, 10000)

In [35]:
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=max_length))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=9, activation='softmax'))

In [36]:
model.compile(loss='categorical_crossentropy',
              optimizer=Nadam(),
              metrics=['accuracy'])

In [40]:
model.fit(X.toarray(), y, batch_size=64, epochs=1, verbose=1, validation_split=.1)

Train on 2988 samples, validate on 333 samples
Epoch 1/1
2988/2988 [==============================] - 3s - loss: 0.6483 - acc: 0.7674 - val_loss: 1.8314 - val_acc: 0.4955


In [24]:
x = vector.transform(test.Text)
x = tran.transform(x)

In [25]:
classes = model.predict(x.toarray())
cols=['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7', 'class8', 'class9']
dfsub = pd.DataFrame(classes, columns=cols)
dfsub.insert(0,'ID',range(len(dfsub.class1)))
dfsub.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,0,0.001379,0.242371,0.000870,0.000676,0.006268,0.001089,0.747008,0.000084,0.000256
1,1,0.359615,0.027254,0.030136,0.465490,0.043340,0.032542,0.029773,0.006848,0.005003
2,2,0.098255,0.273219,0.041841,0.059645,0.072775,0.040148,0.383010,0.010817,0.020291
3,3,0.011393,0.297010,0.006627,0.006135,0.034052,0.011163,0.629005,0.001499,0.003116
4,4,0.262215,0.010692,0.013184,0.669156,0.017245,0.012230,0.012484,0.001790,0.001005


In [26]:
dfsub.to_csv('sknn.csv',index=False)

loss: 0.4416 - acc: 0.8098 - val_loss: 1.5989 - val_acc: 0.4431
submission with this accuracy scored: 1.00164

loss: 0.4624 - acc: 0.8112 - val_loss: 1.2091 - val_acc: 0.6426
submission with this accuracy scored: 0.78966

loss: 1.5341 - acc: 0.4414 - val_loss: 1.3903 - val_acc: 0.5105
submission with this accuracy scored .88833 after 1 epoch of training